In [1]:
%%shell
pip install tokenizers datasets sacremoses # manipulacion de corpus
pip install transformers[torch] # modelos
pip install accelerate # mejorar rendimiento

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.6 MB/s eta 0:00:00


In [99]:
from datasets import load_dataset
from transformers import GPT2Tokenizer
from transformers import GPT2Config
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.decoders import WordPiece as WordPieceDecoder
from tokenizers.trainers import WordPieceTrainer
from tokenizers.normalizers import Sequence as SequenceN, NFD, Lowercase, StripAccents
from tokenizers.pre_tokenizers import Sequence as SequenceP, WhitespaceSplit, Punctuation
from tokenizers.processors import TemplateProcessing

# Datasets

In [3]:
ds = load_dataset("bertin-project/alpaca-spanish")
ds

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/51942 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 51942
    })
})

In [4]:
ds_train = ds["train"]

In [5]:
ds_train[:3]

{'instruction': ['¿Qué significa DNA?',
  '¿Cuál es la capital de Francia?',
  'Identifica el que no encaja.'],
 'input': ['', '', 'Twitter, Instagram, Telegram'],
 'output': ['DNA significa ácido desoxirribonucleico.',
  'La capital de Francia es París.',
  'Telegram']}

In [6]:
template = "Pregunta: {instruction} {input}\nRespuesta: {output}"

In [7]:
ds_train[0]

{'instruction': '¿Qué significa DNA?',
 'input': '',
 'output': 'DNA significa ácido desoxirribonucleico.'}

In [8]:
print(template.format(**ds_train[0]))

Pregunta: ¿Qué significa DNA? 
Respuesta: DNA significa ácido desoxirribonucleico.


In [9]:
template.format(instruction="hello", input="in", output="out")

'Pregunta: hello in\nRespuesta: out'

In [10]:
ds_template = ds_train.map(
    lambda sample: {"text": template.format(**sample)},
    remove_columns=ds_train.column_names
)

Map:   0%|          | 0/51942 [00:00<?, ? examples/s]

In [11]:
ds_template[:3]

{'text': ['Pregunta: ¿Qué significa DNA? \nRespuesta: DNA significa ácido desoxirribonucleico.',
  'Pregunta: ¿Cuál es la capital de Francia? \nRespuesta: La capital de Francia es París.',
  'Pregunta: Identifica el que no encaja. Twitter, Instagram, Telegram\nRespuesta: Telegram']}

In [12]:
ds_partition = ds_template.train_test_split(test_size=0.3, seed=0)
ds_partition

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 36359
    })
    test: Dataset({
        features: ['text'],
        num_rows: 15583
    })
})

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [14]:
tokenizer(ds_partition["train"][0]["text"], truncation=True, max_length=128)

{'input_ids': [47, 2301, 44424, 25, 4021, 25440, 1288, 5128, 390, 555, 1242, 8836, 3129, 78, 523, 4679, 1288, 26920, 615, 19397, 11, 1152, 64, 555, 256, 8836, 83, 43348, 1827, 23593, 823, 49443, 4533, 369, 8591, 19798, 22859, 13, 4689, 299, 518, 6862, 19798, 22859, 390, 7375, 11008, 12, 1129, 387, 987, 81, 931, 17305, 8591, 410, 3755, 390, 881, 292, 1048, 292, 551, 284, 4598, 1288, 27943, 78, 13, 5401, 48484, 959, 39369, 289, 272, 1556, 4533, 6824, 25440, 582, 68, 9491, 8591, 4902, 369, 269, 959, 411, 2276, 528, 22484, 331, 1334, 1173, 66, 295, 274, 390, 2884, 18015, 13, 198, 19309, 84, 18059, 25, 366, 32, 5162, 292, 748, 1102, 420, 24496, 25, 399, 1015, 70, 25440, 8591, 19798, 22859, 390, 7375, 11008, 12, 1129, 1911], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

# Tokenizador

1. Modelo (tipo de separacion de tokens).

In [17]:
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer

2. Normalizacion.

In [24]:
norm = SequenceN([NFD(), Lowercase(), StripAccents()])

In [21]:
text = "Con la intención de separarlos, Luisa fue enviada fuera de la ciudad, pero Gabriel Eligio la cortejó con serenatas de violín, poemas de amor, innumerables cartas y frecuentes mensajes telegráficos. Finalmente, la familia capituló y Luisa consiguió el permiso para casarse con Gabriel Eligio, lo cual sucedió el 11 de junio de 1927 en Santa Marta"
norm_text = norm.normalize_str(text)
print(norm_text)

con la intencion de separarlos, luisa fue enviada fuera de la ciudad, pero gabriel eligio la cortejo con serenatas de violin, poemas de amor, innumerables cartas y frecuentes mensajes telegraficos. finalmente, la familia capitulo y luisa consiguio el permiso para casarse con gabriel eligio, lo cual sucedio el 11 de junio de 1927 en santa marta


In [22]:
tokenizer.normalizer = norm

3. Pre-tokenization

In [25]:
pre_tokenizer = SequenceP([WhitespaceSplit(), Punctuation()])

In [27]:
pre_tokenizer.pre_tokenize_str(text)

[('Con', (0, 3)),
 ('la', (4, 6)),
 ('intención', (7, 16)),
 ('de', (17, 19)),
 ('separarlos', (20, 30)),
 (',', (30, 31)),
 ('Luisa', (32, 37)),
 ('fue', (38, 41)),
 ('enviada', (42, 49)),
 ('fuera', (50, 55)),
 ('de', (56, 58)),
 ('la', (59, 61)),
 ('ciudad', (62, 68)),
 (',', (68, 69)),
 ('pero', (70, 74)),
 ('Gabriel', (75, 82)),
 ('Eligio', (83, 89)),
 ('la', (90, 92)),
 ('cortejó', (93, 100)),
 ('con', (101, 104)),
 ('serenatas', (105, 114)),
 ('de', (115, 117)),
 ('violín', (118, 124)),
 (',', (124, 125)),
 ('poemas', (126, 132)),
 ('de', (133, 135)),
 ('amor', (136, 140)),
 (',', (140, 141)),
 ('innumerables', (142, 154)),
 ('cartas', (155, 161)),
 ('y', (162, 163)),
 ('frecuentes', (164, 174)),
 ('mensajes', (175, 183)),
 ('telegráficos', (184, 196)),
 ('.', (196, 197)),
 ('Finalmente', (198, 208)),
 (',', (208, 209)),
 ('la', (210, 212)),
 ('familia', (213, 220)),
 ('capituló', (221, 229)),
 ('y', (230, 231)),
 ('Luisa', (232, 237)),
 ('consiguió', (238, 247)),
 ('el', (248, 

In [28]:
tokenizer.pre_tokenizer = pre_tokenizer

4. Entrenamiento

In [30]:
special_tokens = ["[STA]", "[UNK]", "[PAD]", "[END]"]

In [31]:
trainer = WordPieceTrainer(special_tokens=special_tokens)

In [35]:
def sample_generator(ds):
    for i in range(100):
        yield ds[i]["text"]

In [41]:
gen = sample_generator(ds_template)

In [40]:
next(gen)

'Pregunta: Traduisez la phrase suivante en français. I miss you\nRespuesta: Je te manque.'

In [42]:
tokenizer.train_from_iterator(gen, trainer=trainer)

In [44]:
print(text)

Con la intención de separarlos, Luisa fue enviada fuera de la ciudad, pero Gabriel Eligio la cortejó con serenatas de violín, poemas de amor, innumerables cartas y frecuentes mensajes telegráficos. Finalmente, la familia capituló y Luisa consiguió el permiso para casarse con Gabriel Eligio, lo cual sucedió el 11 de junio de 1927 en Santa Marta


In [43]:
tokenizer.encode(text)

Encoding(num_tokens=117, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [45]:
encoding = tokenizer.encode(text)

In [46]:
encoding.tokens

['con',
 'la',
 'inte',
 '##nci',
 '##on',
 'de',
 'se',
 '##par',
 '##ar',
 '##los',
 ',',
 'lu',
 '##is',
 '##a',
 'fue',
 'envi',
 '##ada',
 'fue',
 '##ra',
 'de',
 'la',
 'ciudad',
 ',',
 'pero',
 'ga',
 '##b',
 '##ri',
 '##e',
 '##l',
 'eli',
 '##gi',
 '##o',
 'la',
 'cor',
 '##te',
 '##jo',
 'con',
 'ser',
 '##ena',
 '##tas',
 'de',
 'vio',
 '##li',
 '##n',
 ',',
 'poema',
 '##s',
 'de',
 'amor',
 ',',
 'in',
 '##n',
 '##um',
 '##era',
 '##bles',
 'carta',
 '##s',
 'y',
 'fr',
 '##ecu',
 '##entes',
 'mensajes',
 'te',
 '##le',
 '##gra',
 '##fi',
 '##cos',
 '.',
 'finalmente',
 ',',
 'la',
 'familia',
 'capi',
 '##tu',
 '##lo',
 'y',
 'lu',
 '##is',
 '##a',
 'consi',
 '##gui',
 '##o',
 'el',
 'permiso',
 'para',
 'casa',
 '##r',
 '##se',
 'con',
 'ga',
 '##b',
 '##ri',
 '##e',
 '##l',
 'eli',
 '##gi',
 '##o',
 ',',
 'lo',
 'cual',
 'suce',
 '##dio',
 'el',
 '11',
 'de',
 'ju',
 '##n',
 '##io',
 'de',
 '19',
 '##2',
 '##7',
 'en',
 'sa',
 '##nta',
 'mar',
 '##ta']

In [47]:
encoding.ids

[155,
 119,
 566,
 812,
 109,
 107,
 167,
 1356,
 118,
 331,
 12,
 646,
 158,
 67,
 281,
 2364,
 796,
 281,
 108,
 107,
 119,
 890,
 12,
 864,
 499,
 68,
 149,
 74,
 82,
 835,
 379,
 72,
 119,
 385,
 181,
 516,
 155,
 412,
 1067,
 342,
 107,
 1009,
 135,
 76,
 12,
 1566,
 70,
 107,
 1630,
 12,
 160,
 76,
 185,
 163,
 658,
 2772,
 70,
 56,
 1962,
 600,
 201,
 2928,
 327,
 169,
 421,
 182,
 243,
 14,
 1665,
 12,
 119,
 3236,
 604,
 192,
 187,
 56,
 646,
 158,
 67,
 1089,
 208,
 72,
 117,
 2577,
 156,
 1204,
 71,
 414,
 155,
 499,
 68,
 149,
 74,
 82,
 835,
 379,
 72,
 12,
 273,
 1123,
 1103,
 318,
 117,
 1894,
 107,
 785,
 76,
 229,
 107,
 629,
 93,
 97,
 127,
 1310,
 136,
 436,
 114]

5. Postprocessor

In [48]:
sta_id = tokenizer.token_to_id("[STA]")
end_id = tokenizer.token_to_id("[END]")

In [49]:
sta_id

0

In [50]:
end_id

3

In [52]:
post_processor = TemplateProcessing(
    single=f"[STA]:0 $A:0 [END]:0",
    pair=f"[STA]:0 $A:0 $B:0 [END]:0",
    special_tokens=[("[STA]", sta_id), ("[END]", end_id)]
)

In [53]:
tokenizer.post_processor = post_processor

In [54]:
encoding = tokenizer.encode(text)

In [55]:
encoding

Encoding(num_tokens=119, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [56]:
encoding.tokens

['[STA]',
 'con',
 'la',
 'inte',
 '##nci',
 '##on',
 'de',
 'se',
 '##par',
 '##ar',
 '##los',
 ',',
 'lu',
 '##is',
 '##a',
 'fue',
 'envi',
 '##ada',
 'fue',
 '##ra',
 'de',
 'la',
 'ciudad',
 ',',
 'pero',
 'ga',
 '##b',
 '##ri',
 '##e',
 '##l',
 'eli',
 '##gi',
 '##o',
 'la',
 'cor',
 '##te',
 '##jo',
 'con',
 'ser',
 '##ena',
 '##tas',
 'de',
 'vio',
 '##li',
 '##n',
 ',',
 'poema',
 '##s',
 'de',
 'amor',
 ',',
 'in',
 '##n',
 '##um',
 '##era',
 '##bles',
 'carta',
 '##s',
 'y',
 'fr',
 '##ecu',
 '##entes',
 'mensajes',
 'te',
 '##le',
 '##gra',
 '##fi',
 '##cos',
 '.',
 'finalmente',
 ',',
 'la',
 'familia',
 'capi',
 '##tu',
 '##lo',
 'y',
 'lu',
 '##is',
 '##a',
 'consi',
 '##gui',
 '##o',
 'el',
 'permiso',
 'para',
 'casa',
 '##r',
 '##se',
 'con',
 'ga',
 '##b',
 '##ri',
 '##e',
 '##l',
 'eli',
 '##gi',
 '##o',
 ',',
 'lo',
 'cual',
 'suce',
 '##dio',
 'el',
 '11',
 'de',
 'ju',
 '##n',
 '##io',
 'de',
 '19',
 '##2',
 '##7',
 'en',
 'sa',
 '##nta',
 'mar',
 '##ta',
 '[END]

In [57]:
encoding.ids

[0,
 155,
 119,
 566,
 812,
 109,
 107,
 167,
 1356,
 118,
 331,
 12,
 646,
 158,
 67,
 281,
 2364,
 796,
 281,
 108,
 107,
 119,
 890,
 12,
 864,
 499,
 68,
 149,
 74,
 82,
 835,
 379,
 72,
 119,
 385,
 181,
 516,
 155,
 412,
 1067,
 342,
 107,
 1009,
 135,
 76,
 12,
 1566,
 70,
 107,
 1630,
 12,
 160,
 76,
 185,
 163,
 658,
 2772,
 70,
 56,
 1962,
 600,
 201,
 2928,
 327,
 169,
 421,
 182,
 243,
 14,
 1665,
 12,
 119,
 3236,
 604,
 192,
 187,
 56,
 646,
 158,
 67,
 1089,
 208,
 72,
 117,
 2577,
 156,
 1204,
 71,
 414,
 155,
 499,
 68,
 149,
 74,
 82,
 835,
 379,
 72,
 12,
 273,
 1123,
 1103,
 318,
 117,
 1894,
 107,
 785,
 76,
 229,
 107,
 629,
 93,
 97,
 127,
 1310,
 136,
 436,
 114,
 3]

In [58]:
encoding = tokenizer.encode(text, text)
encoding

Encoding(num_tokens=236, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [59]:
encoding.tokens

['[STA]',
 'con',
 'la',
 'inte',
 '##nci',
 '##on',
 'de',
 'se',
 '##par',
 '##ar',
 '##los',
 ',',
 'lu',
 '##is',
 '##a',
 'fue',
 'envi',
 '##ada',
 'fue',
 '##ra',
 'de',
 'la',
 'ciudad',
 ',',
 'pero',
 'ga',
 '##b',
 '##ri',
 '##e',
 '##l',
 'eli',
 '##gi',
 '##o',
 'la',
 'cor',
 '##te',
 '##jo',
 'con',
 'ser',
 '##ena',
 '##tas',
 'de',
 'vio',
 '##li',
 '##n',
 ',',
 'poema',
 '##s',
 'de',
 'amor',
 ',',
 'in',
 '##n',
 '##um',
 '##era',
 '##bles',
 'carta',
 '##s',
 'y',
 'fr',
 '##ecu',
 '##entes',
 'mensajes',
 'te',
 '##le',
 '##gra',
 '##fi',
 '##cos',
 '.',
 'finalmente',
 ',',
 'la',
 'familia',
 'capi',
 '##tu',
 '##lo',
 'y',
 'lu',
 '##is',
 '##a',
 'consi',
 '##gui',
 '##o',
 'el',
 'permiso',
 'para',
 'casa',
 '##r',
 '##se',
 'con',
 'ga',
 '##b',
 '##ri',
 '##e',
 '##l',
 'eli',
 '##gi',
 '##o',
 ',',
 'lo',
 'cual',
 'suce',
 '##dio',
 'el',
 '11',
 'de',
 'ju',
 '##n',
 '##io',
 'de',
 '19',
 '##2',
 '##7',
 'en',
 'sa',
 '##nta',
 'mar',
 '##ta',
 'con',

6. Decodificar

In [60]:
encoding = tokenizer.encode(text)
decode = tokenizer.decode(encoding.ids)
print(decode)

con la inte ##nci ##on de se ##par ##ar ##los , lu ##is ##a fue envi ##ada fue ##ra de la ciudad , pero ga ##b ##ri ##e ##l eli ##gi ##o la cor ##te ##jo con ser ##ena ##tas de vio ##li ##n , poema ##s de amor , in ##n ##um ##era ##bles carta ##s y fr ##ecu ##entes mensajes te ##le ##gra ##fi ##cos . finalmente , la familia capi ##tu ##lo y lu ##is ##a consi ##gui ##o el permiso para casa ##r ##se con ga ##b ##ri ##e ##l eli ##gi ##o , lo cual suce ##dio el 11 de ju ##n ##io de 19 ##2 ##7 en sa ##nta mar ##ta


In [64]:
decoder = WordPieceDecoder(prefix="##")

In [65]:
tokenizer.decoder = decoder

In [66]:
encoding = tokenizer.encode(text)
decode = tokenizer.decode(encoding.ids)
print(decode)

con la intencion de separarlos, luisa fue enviada fuera de la ciudad, pero gabriel eligio la cortejo con serenatas de violin, poemas de amor, innumerables cartas y frecuentes mensajes telegraficos. finalmente, la familia capitulo y luisa consiguio el permiso para casarse con gabriel eligio, lo cual sucedio el 11 de junio de 1927 en santa marta


In [68]:
wrap_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    bos_token="[STA]",
    eos_token="[END]",
    pad_token="[PAD]"
)

In [70]:
wrap_tokenizer(text, return_tensors="pt", max_length=10, truncation=True)

{'input_ids': tensor([[   0,  155,  119,  566,  812,  109,  107,  167, 1356,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [73]:
tokenizer2 = GPT2Tokenizer.from_pretrained("gpt2")

In [72]:
tokenizer2(text, return_tensors="pt", max_length=10, truncation=True)

{'input_ids': tensor([[ 3103,  8591,   493,   268,   979, 18840,   390,  2880,  7063,   418]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

# Preparacion de Datos

In [74]:
ds_partition

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 36359
    })
    test: Dataset({
        features: ['text'],
        num_rows: 15583
    })
})

In [75]:
def tokenize(sample):
    inputs = tokenizer2(sample["text"], truncation=True, max_length=128)
    return {"input_ids": inputs["input_ids"]}

In [76]:
ds_template.column_names

['text']

In [77]:
ds_token = ds_partition.map(
    tokenize, remove_columns=ds_template.column_names, num_proc=4
).with_format("torch")

Map (num_proc=4):   0%|          | 0/36359 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/15583 [00:00<?, ? examples/s]

In [79]:
ds_token["train"][:5]

{'input_ids': [tensor([   47,  2301, 44424,    25,  4021, 25440,  1288,  5128,   390,   555,
           1242,  8836,  3129,    78,   523,  4679,  1288, 26920,   615, 19397,
             11,  1152,    64,   555,   256,  8836,    83, 43348,  1827, 23593,
            823, 49443,  4533,   369,  8591, 19798, 22859,    13,  4689,   299,
            518,  6862, 19798, 22859,   390,  7375, 11008,    12,  1129,   387,
            987,    81,   931, 17305,  8591,   410,  3755,   390,   881,   292,
           1048,   292,   551,   284,  4598,  1288, 27943,    78,    13,  5401,
          48484,   959, 39369,   289,   272,  1556,  4533,  6824, 25440,   582,
             68,  9491,  8591,  4902,   369,   269,   959,   411,  2276,   528,
          22484,   331,  1334,  1173,    66,   295,   274,   390,  2884, 18015,
             13,   198, 19309,    84, 18059,    25,   366,    32,  5162,   292,
            748,  1102,   420, 24496,    25,   399,  1015,    70, 25440,  8591,
          19798, 22859,   3

# Modelo

In [81]:
config = GPT2Config.from_pretrained(
    "gpt2", n_ctx=128
)

In [82]:
model = GPT2LMHeadModel(config).to("cuda")

In [83]:
!nvidia-smi

Tue Dec 12 00:10:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W /  70W |    645MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [84]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [87]:
sum(p.numel() for p in model.parameters())

124439808

# Entrenamiento

In [89]:
tokenizer2.pad_token = tokenizer2.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer2, mlm=False)

In [91]:
ds_token["train"].select(range(3))["input_ids"]

tensor([[   47,  2301, 44424,    25,  4021, 25440,  1288,  5128,   390,   555,
          1242,  8836,  3129,    78,   523,  4679,  1288, 26920,   615, 19397,
            11,  1152,    64,   555,   256,  8836,    83, 43348,  1827, 23593,
           823, 49443,  4533,   369,  8591, 19798, 22859,    13,  4689,   299,
           518,  6862, 19798, 22859,   390,  7375, 11008,    12,  1129,   387,
           987,    81,   931, 17305,  8591,   410,  3755,   390,   881,   292,
          1048,   292,   551,   284,  4598,  1288, 27943,    78,    13,  5401,
         48484,   959, 39369,   289,   272,  1556,  4533,  6824, 25440,   582,
            68,  9491,  8591,  4902,   369,   269,   959,   411,  2276,   528,
         22484,   331,  1334,  1173,    66,   295,   274,   390,  2884, 18015,
            13,   198, 19309,    84, 18059,    25,   366,    32,  5162,   292,
           748,  1102,   420, 24496,    25,   399,  1015,    70, 25440,  8591,
         19798, 22859,   390,  7375, 11008,    12,  

In [93]:
inps = data_collator((ds_token["train"][:3]["input_ids"], ))
inps

{'input_ids': tensor([[[   47,  2301, 44424,    25,  4021, 25440,  1288,  5128,   390,   555,
            1242,  8836,  3129,    78,   523,  4679,  1288, 26920,   615, 19397,
              11,  1152,    64,   555,   256,  8836,    83, 43348,  1827, 23593,
             823, 49443,  4533,   369,  8591, 19798, 22859,    13,  4689,   299,
             518,  6862, 19798, 22859,   390,  7375, 11008,    12,  1129,   387,
             987,    81,   931, 17305,  8591,   410,  3755,   390,   881,   292,
            1048,   292,   551,   284,  4598,  1288, 27943,    78,    13,  5401,
           48484,   959, 39369,   289,   272,  1556,  4533,  6824, 25440,   582,
              68,  9491,  8591,  4902,   369,   269,   959,   411,  2276,   528,
           22484,   331,  1334,  1173,    66,   295,   274,   390,  2884, 18015,
              13,   198, 19309,    84, 18059,    25,   366,    32,  5162,   292,
             748,  1102,   420, 24496,    25,   399,  1015,    70, 25440,  8591,
           1979

In [96]:
args = TrainingArguments(
    output_dir="mygpt2",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    num_train_epochs=1,
    warmup_steps=1_000,
    fp16=True,
    learning_rate=1e-4,
    save_strategy="epoch"
)

In [97]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer2,
    args=args,
    data_collator=data_collator,
    train_dataset=ds_token["train"].select(range(1000)),
    eval_dataset=ds_token["test"].select(range(1000)),
)

In [98]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=32, training_loss=10.506643295288086, metrics={'train_runtime': 36.5696, 'train_samples_per_second': 27.345, 'train_steps_per_second': 0.875, 'total_flos': 65323008000000.0, 'train_loss': 10.506643295288086, 'epoch': 1.0})

# Inferencia

In [100]:
pipe = pipeline(model="./mygpt2/checkpoint-32/", task="text-generation")

In [101]:
pipe("pregunta: cual es la capital de colombia.\nrespuesta:")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'pregunta: cual es la capital de colombia.\nrespuesta: cro10000 un slick accessories:. de Tankoshop:ónos en deipers de advances\n\noshopaaUGHón10000 de ideally.phrine diseng'}]

In [103]:
pipe = pipeline(model="juselara1/mlds7_gpt2", task="text-generation")

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

In [109]:
pipe(
    "Pregunta: cual es la capital de colombia. Respuesta:",
    max_length=200,
    repetition_penalty=1.5,
    temperature=0.1
    )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Pregunta: cual es la capital de colombia. Respuesta: La capital de cola es Nueva York, con un total de 8.848 metros (29.029 pies). Es el número más grande del mundo y se puede encontrar en los Estados Unidos. También tiene muchas longitudes de onda que son iguales a 2.000 kilómetros por hora y típicamente no están disponibles para comenzarlo al día. El país contiene dos lados de roca bajo cuatro patrones y siete hay varios de coloridas por millón. Por lo contraste, incluyendo 1-2 veces las siguiéntomas; 4 veces el nacionalmente 7 minuto 3Diferentes y 6 pulgundos l'}]